<a href="https://colab.research.google.com/github/MohneetKaur/CNN-using-GoogleColab/blob/main/Optimize_CNN_model_using_keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 11.3 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.12.0


In [4]:
fashion_mnist = keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [6]:
train_images = train_images/255.0
test_images = test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
len(train_images)

60000

In [9]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [10]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

  return model

In [17]:
from keras_tuner import RandomSearch
from kerastuner.engine.hyperparameters import hyperparameters

In [18]:
tuner_search = RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name='Mnist Fashion')

In [19]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 31s]
val_accuracy: 0.906499981880188

Best val_accuracy So Far: 0.909333348274231
Total elapsed time: 00h 02m 39s


In [20]:
model = tuner_search.get_best_models(num_models=1)[0]

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 112)       1120      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 32)        89632     
                                                                 
 flatten (Flatten)           (None, 15488)             0         
                                                                 
 dense (Dense)               (None, 80)                1239120   
                                                                 
 dense_1 (Dense)             (None, 10)                810       
                                                                 
Total params: 1,330,682
Trainable params: 1,330,682
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 10s 5ms/step - loss: 0.1466 - accuracy: 0.9451 - val_loss: 0.2827 - val_accuracy: 0.9058
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1103 - accuracy: 0.9590 - val_loss: 0.2954 - val_accuracy: 0.9108
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0806 - accuracy: 0.9698 - val_loss: 0.3563 - val_accuracy: 0.9090
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0585 - accuracy: 0.9791 - val_loss: 0.3456 - val_accuracy: 0.9150
Epoch 8/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0472 - accuracy: 0.9821 - val_loss: 0.4451 - val_accuracy: 0.9093
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0380 - accuracy: 0.9861 - val_loss: 0.4723 - val_accuracy: 0.9073
Epoch 10/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0280 - accuracy: 0.9899 - val_loss: 0.4889 - val_accura